In [438]:
import numpy as np

class gridWorld(object):
    def __init__(self, shape = (5, 5)):
        self.shape = shape
        self.rewards = [[-1 for x in range(shape[0])] for y in range(shape[1])]
        self.board_mask = [[0 for x in range(shape[0])] for y in range(shape[1])]
        self.x_pos = 0
        self.y_pos = 0
        
    def actions(self, state):
        if state == (4, 4):
            return []
        return ['U', 'D', 'L', 'R']
    
    def states(self):
        for x in range(self.shape[0]):
            for y in range(self.shape[1]):
                if self.board_mask[x][y] == 0:
                    yield (x, y)
    
    def reward(self, state):
        return self.rewards[state[0]][state[1]]
    
    def probability(self, s_next, s, a):
        x = s[0]
        y = s[1]
        if(a == 'D'):
            y = y + 1
        elif(a == 'U'):
            y = y - 1
        elif(a == 'L'):
            x = x - 1
        elif(a == 'R'):
            x = x + 1
        if (x in range(0, self.shape[0])) and (y in range(0, self.shape[1])) and (self.board_mask[x][y] == 0):
            if (s_next == (x, y)):
                return 1
            return 0
        elif s_next == s:
            return 1
        return 0
                
        
    
    def step(self, action):
        x = self.x_pos
        y = self.y_pos
        if(action == 'D'):
            y = y + 1
        elif(action == 'U'):
            y = y - 1
        elif(action == 'L'):
            x = x - 1
        elif(action == 'R'):
            x = x + 1
        else:
            return
        if (x in range(0, self.shape[0])) and (y in range(0, self.shape[1])) and (self.board_mask[x][y] == 0):
            self.x_pos = x
            self.y_pos = y
        
    def render(self):
        for y in range(self.shape[1]):
            for x in range(self.shape[0]):
                if x == self.x_pos and y == self.y_pos:
                    print('x', end='\t')
                else:
                    print(self.board_mask[x][y], end='\t')
            print('')

In [439]:
env = gridWorld()
env.board_mask[4][3] = 1
env.board_mask[3][3] = 1
env.board_mask[2][3] = 1
env.board_mask[1][3] = 1
env.board_mask[0][1] = 1
env.board_mask[1][1] = 1
env.rewards[4][4] = 1
env.render()

x	0	0	0	0	
1	1	0	0	0	
0	0	0	0	0	
0	1	1	1	1	
0	0	0	0	0	


In [414]:
import copy
import numpy as np
def value_iteration(mdp, epsilon, gamma):
    V = dict()
    # Initialize utilities to zero
    for s in env.states():
        V.update({s: 0.0})
    while(True):
        V_prev = copy.deepcopy(V)
        delta = 0
        for s in mdp.states():
            lst = []
            for a in mdp.actions(s):
                for s_next in mdp.states():
                    if mdp.probability(s_next, s, a) > 0:
                        lst.append(mdp.probability(s_next, s, a)*V_prev[s_next])
            V[s] = mdp.reward(s) + (0 if (lst == []) else gamma*np.max(lst))
            if np.abs(V[s] - V_prev[s]) > delta:
                delta = np.abs(V[s] - V_prev[s])
        #print(delta)
        if delta < epsilon*(1 - gamma)/gamma:
            return V
        
def value_iteration(mdp, epsilon, gamma):
    
        
def policy(mdp, V):
    PI = dict()
    for s in mdp.states():
        a_lst = []
        v_lst = []
        for a in mdp.actions(s):
            for s_next in mdp.states():
                if mdp.probability(s_next, s, a) > 0:
                    v_lst.append(mdp.probability(s_next, s, a)*V[s_next])
                    a_lst.append(a)
        PI.update({s: a_lst[np.argmax(v_lst)]})
    return PI
            
                        

In [441]:
V = value_iteration(env, 1e-3, 0.9)
#P = policy(env, V)

for y in range(5):
    for x in range(5):
        try:
            #print(P[(x, y)], end = '\t')
            print('{0:.1f}'.format(V[(x, y)]), end='\t')
        except:
            print('x', end='\t')
    print('')

-6.9	-6.5	-6.2	-6.5	-6.9	
x	x	-5.7	-6.2	-6.5	
-4.2	-4.7	-5.3	-5.7	-6.2	
-3.5	x	x	x	x	
-2.8	-2.0	-1.1	-0.1	1.0	


In [435]:
s = (0, 0)
#[(p, a) for p in [(s, s_next, a, env.probability(s_next, s, a)*V[s_next]) for s_next in env.states()] for a in env.actions(s)]

a_lst = []
v_lst = []
for a in env.actions(s):
    for s_next in env.states():
        if env.probability(s_next, s, a) > 0:
            v_lst.append(env.probability(s_next, s, a)*V[s_next])
            a_lst.append(a)
            print(s, s_next, a, env.probability(s_next, s, a), V[s_next])
        
#lst
    

(0, 0) (0, 0) U 1 -10.99999946
(0, 0) (0, 0) D 1 -10.99999946
(0, 0) (0, 0) L 1 -10.99999946
(0, 0) (1, 0) R 0.7 -9.99999956


In [445]:
# snake game in python

import pygame, sys, time, random

# check for initialising error
check_error = pygame.init()
if check_error[1] > 0:
    print("(!) Had {0} initializing errors, exiting...".format(check_error[1]))
    sys.exit(-1)
else:
    print("(+) PyGame successfully initialized")

# Play Surface
playSurface = pygame.display.set_mode((720, 460))
pygame.display.set_caption('Snake Game ----:>')
#time.sleep(5)

# Colors
red = pygame.Color(255,0,0)#gameOver
green = pygame.Color(0,255,0)#Snake
black = pygame.Color(0,0,0)#Score
white = pygame.Color(255,255,255)#background
brown = pygame.Color(162,42,42)#food

# FPS Controller
fpsController = pygame.time.Clock()

# Important Variables
snakePos = [100,50]
snakeBody = [[100,50],[90,50],[80,50]] #,[70,50],[60,50],[50,50],[40,50],[30,50],[20,50]

foodPos = [random.randrange(1,72)*10, random.randrange(1,46)*10]
foodSpawn = True

direction = 'RIGHT'
changeTo = direction

score = 0

#Game Over Function
def gameOver():
    myFont = pygame.font.SysFont('monaco', 72)
    GOsurf = myFont.render('Game Over!', True, red)
    GOrect = GOsurf.get_rect()
    GOrect.midtop = (360,15)
    playSurface.blit(GOsurf, GOrect)
    showScore(0)
    pygame.display.flip()
    time.sleep(2)
    pygame.quit()
    sys.exit()

# score function
def showScore(choice=1):
    sFont = pygame.font.SysFont('monaco', 24)
    Ssurf = sFont.render('Score : {0}'.format(score), True, black)
    Srect = Ssurf.get_rect()
    if choice ==1:
        Srect.midtop = (80, 10)
    else:
        Srect.midtop = (360, 150)

    playSurface.blit(Ssurf, Srect)

#Main Logic Of Game
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_RIGHT or event.key == ord('d'):
                changeTo = 'RIGHT'
            if event.key == pygame.K_LEFT or event.key == ord('a'):
                changeTo = 'LEFT'
            if event.key == pygame.K_UP or event.key == ord('w'):
                changeTo = 'UP'
            if event.key == pygame.K_DOWN or event.key == ord('s'):
                changeTo = 'DOWN'
            if event.key == pygame.K_ESCAPE:
                pygame.event.post(pygame.event.Event(pygame.QUIT))

    # Validation of direction
    if changeTo == 'RIGHT' and not direction == 'LEFT':
        direction = 'RIGHT'
    if changeTo == 'LEFT' and not direction == 'RIGHT':
        direction = 'LEFT'
    if changeTo == 'UP' and not direction == 'DOWN':
        direction = 'UP'
    if changeTo == 'DOWN' and not direction == 'UP':
        direction = 'DOWN'

    if direction == 'RIGHT':
        snakePos[0] += 10
    if direction == 'LEFT':
        snakePos[0] -= 10
    if direction == 'UP':
        snakePos[1] -= 10
    if direction == 'DOWN':
        snakePos[1] += 10

    # snake bodyh mechanism
    snakeBody.insert(0, list(snakePos))
    if snakePos[0] == foodPos[0] and snakePos[1] == foodPos[1]:
        score+=10
        foodSpawn = False
    else:
        snakeBody.pop()

    if foodSpawn == False:
        foodPos = [random.randrange(1, 72) * 10, random.randrange(1, 46) * 10]
    foodSpawn = True

    playSurface.fill(white)
    for pos in snakeBody:
        pygame.draw.rect(playSurface,green,pygame.Rect(pos[0], pos[1],10,10))

    pygame.draw.rect(playSurface, brown, pygame.Rect(foodPos[0], foodPos[1], 10, 10))

    if snakePos[0] > 710 or snakePos[0] < 0:
        gameOver()
    if snakePos[1] > 450 or snakePos[1] < 0:
        gameOver()

    for block in snakeBody[1:]:
        if snakePos[0] == block[0] and snakePos[1] == block[1]:
            gameOver()

    showScore()
    pygame.display.flip()
    fpsController.tick(10)

(+) PyGame successfully initialized


SystemExit: 

C:\python\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
